In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
# 设置gpu内存自增长
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 数据处理

In [3]:
input_filepath = './shakespeare.txt'
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# 1. 生成词表
# 2. 建立映射 char -> id
# 3. data -> id
# 4. 定义模型的输入和输出

# 1. 生成词表
vocab = sorted(set(text))  # 新建一个set对象, set会将重复的字符去掉因此可以用来当做词表
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# 2. 建立映射 char -> id
char2idx = {char: idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
# 3. data -> id
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
# 4. 定义模型的输入和输出
def split_input_target(id_text):
    return id_text[0:-1], id_text[1:]

# 字符数据集
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
# 等长分割(长度为seq_length)，不是按照完整的句子/单词结构
# 句子数据集
seq_dataset = char_dataset.batch(seq_length + 1, drop_remainder = True)  # drop_remainder=True,batch做到最后多出来的数据丢掉

for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
# 拆分输入和输出
seq_dataset = seq_dataset.map(split_input_target)
for item_input, item_output in seq_dataset.take(1):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]


## 构建模型

In [10]:
batch_size = 64
buffer_size = 10000
seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [11]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(units=rnn_units, 
                          stateful = True,
                          recurrent_initializer = 'glorot_uniform',
                          return_sequences=True),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_batch, target_batch in seq_dataset.take(1):
    batch_prediction = model(input_batch)
    print(batch_prediction.shape)

(64, 100, 65)


In [13]:
# 随机采样
# 采用概率最大的字符作为预测值：贪心策略(greedy)，只能产生一个序列；采用随机采样的方式来输出预测值：随机策略(random),能产生多个序列
sample_indices = tf.random.categorical(logits = batch_prediction[0], # batch_prediction[0]: [100, 65]  # logits：对于分类任务来说，计算softmax之前的值成为logits
                                       num_samples = 1   # 随机采样几个数
                                       )
print(sample_indices.shape)
sample_indices = tf.squeeze(sample_indices, axis=-1)  # 去掉一个多余的维度
print(sample_indices.shape)

(100, 1)
(100,)


In [14]:
print("Input: ", repr("".join(idx2char[input_batch[0]])))
print("Output: ", repr("".join(idx2char[target_batch[0]])))
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  'nap.\nBut did I never speak of all that time?\n\nFirst Servant:\nO, yes, my lord, but very idle words:\nF'
Output:  'ap.\nBut did I never speak of all that time?\n\nFirst Servant:\nO, yes, my lord, but very idle words:\nFo'
Predictions:  'O,OYZvtbIYYfzLuU?D?tfxCHOQV&qOIyWWxU3QCGECI;ynrjEFniJ3ZnENms3t.p-O!FQpYDbgUD.Ft-hlwAm-&D-XOotYmMgoyb'


In [15]:
# 自定义损失函数
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)  # 默认是以概率分布(经过softmax激活函数)和labels为输入,在这里以logits作为输入，因此要设from_logits=True

model.compile(optimizer='adam', loss=loss)
example_loss = loss(target_batch, batch_prediction)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1748257


In [16]:
# 训练模型 将模型保存下来
output_dir = './text_generation_checkpoints'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
epochs = 100
history = model.fit(seq_dataset, epochs=epochs, callbacks=[checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 13s 74ms/step - loss: 2.5782
Epoch 2/100
172/172 [==============================] - 11s 61ms/step - loss: 1.8777
Epoch 3/100
172/172 [==============================] - 10s 61ms/step - loss: 1.6276
Epoch 4/100
172/172 [==============================] - 11s 61ms/step - loss: 1.4955
Epoch 5/100
172/172 [==============================] - 11s 61ms/step - loss: 1.4174
Epoch 6/100
172/172 [==============================] - 11s 62ms/step - loss: 1.3603
Epoch 7/100
172/172 [==============================] - 11s 62ms/step - loss: 1.3164
Epoch 8/100
172/172 [==============================] - 11s 62ms/step - loss: 1.2787
Epoch 9/100
172/172 [==============================] - 11s 62ms/step - loss: 1.2426
Epoch 10/100
172/172 [==============================] - 11s 62ms/step - loss: 1.2088
Epoch 11/100
172/172 [==============================] - 11s 62ms/step - loss: 1.1727
Epoch 12/100
172/172 [==============================] - 11s 61ms/step - lo

172/172 [==============================] - 11s 62ms/step - loss: 0.3871
Epoch 98/100
172/172 [==============================] - 11s 62ms/step - loss: 0.3857
Epoch 99/100
172/172 [==============================] - 11s 62ms/step - loss: 0.3865
Epoch 100/100
172/172 [==============================] - 11s 62ms/step - loss: 0.3843


In [17]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints\\ckpt_100'

In [18]:
# 有checkpoint载入模型  用于预测
model2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)   # batch_size=1: 预测时一次只生成一个句子
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None])) # 设置输入的size, 1表示输入一个样本，None指输入变长的句子
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 文本生成
# start char sequence A,
# A -> model -> b
# A.append(b) -> B
# B -> model -> c
# B.append(c) -> C
# ...
def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)  # 增加一个维度，因为模型的输入是[1, None]
    text_generated = []  # 用于存放生成的字符
    model.reset_states()
    
    # temperature > 1, 概率分布更加均匀，更接近random
    # temperature < 1, 概率分布更加陡峭，更加紧greedy
    temperature = 0.5
    for _ in range(num_generate):
        # 1. input_eval model ingerence -> predictions
        # 2. sample -> char ->text_generate
        # 3. update input_eval
        # predictions: [batch_size, len(input_eval), vocab_size]
        predictions = model(input_eval)
        predictions = predictions / temperature  # 此处predictions是logits，当除以一个小于1值时，在经过softmax后，概率分布会变陡峭，也就更加接近greedy算法
        # predictions: [batch_size, len(input_eval), vocab_size] -> [len(input_eval), vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [len(input_eval, 1)]
        predicted_ids = tf.random.categorical(predictions, num_samples=1)
        predicted_id = predicted_ids[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y 只需要一个字符作为输入
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)
new_text = generate_text(model2, 'All: ')
print(new_text)

All: belike he me,
And made her widow to a borrow that the haste strait'st that bear the corse, and set it down.

LADY ANNE:
Would it were mortal policy, hate the hour that e'er I undertake the traitor for thy faults.

TRANIO:
O tis sweet with my native promise
To excels the child: I not a maid we should be cold.

GLOUCESTER:
Look, how they come to him their friends at peace.

BUCKINGHAM:
What thinks your daughter of my words,
Break our spite, and doth beg the all-cherits of a wilderness,
And spit upon him, whilst I do beseech your grace
Look on my wrongs did before Corioli loured the north,
When they should see thee here, faint and not aver she was bornt it.

VINCENTIO:
Come hither, you rogue. What, have you forgot me?

BIONDELLO:
Faith, nothing; but has let them take my leave of you, and put up my mind.

DUCHESS OF YORK:
A god on earth thou art, bound from mistaking,--

BUCKINGHAM:
No, by my troth, if I be good of you.

Roman:
'Tis well, sir, what an if
The foul that hath overtages h